In [7]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier #의사결정 알고리즘 지원
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score #평가 Classification, Regression, Clustering 등 성능 측정 방법 제공


iris = load_iris() # Iris 데이터 세트 불러오기


dt_clf = DecisionTreeClassifier()       # Classification에 사용할 모델 불러오기
train_data = iris.data                  # 학습에 사용할 Train 데이터 저장
train_label = iris.target               # 학습에 사용할 Train Label 저장 iris의 종류를 정해주는 것
dt_clf.fit(train_data, train_label)     # 학습 진행

pred = dt_clf.predict(train_data)       # 학습된 모델을 검증
print("정확도(accuracy) : ", accuracy_score(train_label, pred))    # 정확도 출력

정확도(accuracy) :  1.0


사이킷런 익히기
sklearn.datasets : 예제 데이터 세트 제공함
### 여기선 Iris 데이터 셋을 제공
ML 알고리즘
sklearn.ensemble : 앙상블 알고리즘 제공
sklearn.linear_model : 선형 회귀 및 로지스틱 회귀 등 Regression 관련 알고리즘 지원
sklearn.naive_bayes : 나이브 베이즈 알고리즘 제공
sklearn.neighbors : 최근접 이웃 알고리즘 제공
sklearn.svm : Support Vector Machine 알고리즘 제공
sklearn.tree : 의사 결정 트리 알고리즘 제공
sklearn.cluster : 비지도 클러스터링 (Unsupervised Clustering) 알고리즘 제공
각종 모델을 사용

사용할 모델.fit(train데이터, train종류들)
pred = 사용할 모델.predict(train데이터)

정확도 검증 : sklearn.metrics(train종류들, pred)

In [16]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

iris = load_iris()
data_clf = DecisionTreeClassifier()
data_clf.fit(iris.data,iris.target)
pred = data_clf.predict(iris.data)
print(accuracy_score(iris.target,pred)*100)
#위와같은 정확도는 전체데이터를 검증데이터로 사용하므로 정확도가 모두같은 100% 확률이 나온다

X_train, X_test, Y_train, Y_test = train_test_split(iris.data,pred,test_size = 0.25) 
data_clf.fit(X_train, Y_train) #데이터들을 data_clf모델로 학습 진행
pred = data_clf.predict(X_test) #위의 학습한 데이터를 성능 평가를위해 pred리스트에 삽입

print(accuracy_score(Y_test,pred))

100.0
0.9210526315789473


train_test_split(arrays, test_size, train_size, random_state, shuffle, stratify)
arrays : 데이터 입력
test_size : Test data의 비율 (default = 0.25)
train_size : Train data의 비율 (default = 나머지 데이터)
random_state : shuffle을 위한 시드값
shuffle : shuffle 여부 (default = True)
stratify : Dataset의 비율 유지